In [2]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)


In [3]:
import pandas as pd

# Path to your local metadata file
metadata_file_path = "meta.xlsx"

# Read Excel file
metadata_df = pd.read_excel(metadata_file_path, engine='openpyxl', sheet_name="Metadata")

# Preview Metadata
print(metadata_df.head())

              Company name Document Name Company ID (Ticker) Document ID Type  \
0                 Reliance    RIL_AR_O&G        INE002A01018               AR   
1                 Reliance    RIL_IP_O&G        INE002A01018               IP   
2                 Reliance    RIL_SR_O&G        INE002A01018               SR   
3                 Reliance   RIL_ECT_O&G        INE002A01018              ECT   
4  Bharat Petroleum (BPCL)   BPCL_AR_O&G        INE029A01011               AR   

  Document Category ID  
0                  FIN  
1                  INV  
2                  ESG  
3               SCRIPT  
4                  FIN  


In [4]:
# Add ".pdf" to each document name
metadata_df['file_name'] = metadata_df['Document Name'] + ".pdf"

# Convert metadata to a dictionary with file_name as the key
metadata_dict = metadata_df.set_index('file_name').to_dict(orient="index")

# Print metadata for each document
for file_name, meta in metadata_dict.items():
    print(f"Document: {file_name}, Metadata: {meta}")


Document: RIL_AR_O&G.pdf, Metadata: {'Company name': 'Reliance', 'Document Name': 'RIL_AR_O&G', 'Company ID (Ticker)': 'INE002A01018', 'Document ID Type': 'AR', 'Document Category ID': 'FIN'}
Document: RIL_IP_O&G.pdf, Metadata: {'Company name': 'Reliance', 'Document Name': 'RIL_IP_O&G', 'Company ID (Ticker)': 'INE002A01018', 'Document ID Type': 'IP', 'Document Category ID': 'INV'}
Document: RIL_SR_O&G.pdf, Metadata: {'Company name': 'Reliance', 'Document Name': 'RIL_SR_O&G', 'Company ID (Ticker)': 'INE002A01018', 'Document ID Type': 'SR', 'Document Category ID': 'ESG'}
Document: RIL_ECT_O&G.pdf, Metadata: {'Company name': 'Reliance', 'Document Name': 'RIL_ECT_O&G', 'Company ID (Ticker)': 'INE002A01018', 'Document ID Type': 'ECT', 'Document Category ID': 'SCRIPT'}
Document: BPCL_AR_O&G.pdf, Metadata: {'Company name': 'Bharat Petroleum (BPCL)', 'Document Name': 'BPCL_AR_O&G', 'Company ID (Ticker)': 'INE029A01011', 'Document ID Type': 'AR', 'Document Category ID': 'FIN'}
Document: BPCL_IP

In [5]:
#techgeniusbucket
import boto3

# Initialize S3 client
s3_client = boto3.client('s3')

# Define S3 bucket where PDFs are stored
s3_bucket = "techgeniusbucket"

# List all PDFs in S3
def list_pdfs_in_s3(bucket_name):
    pdf_files = []
    response = s3_client.list_objects_v2(Bucket=bucket_name)

    for obj in response.get('Contents', []):
        if obj['Key'].endswith(".pdf"):
            pdf_files.append(obj['Key'])

    return pdf_files

# Fetch all PDFs in S3
pdf_files = list_pdfs_in_s3(s3_bucket)
print("PDF Files in S3:", pdf_files)


PDF Files in S3: ['ABA_AR_O&G.pdf', 'ADA_AR_O&G.pdf', 'ADA_ECT_O&G.pdf', 'ADA_IP_O&G.pdf', 'ADA_SR_O&G.pdf', 'BPCL_AR_O&G.pdf', 'BPCL_ECT_O&G.pdf', 'BPCL_IP_O&G.pdf', 'BPCL_SR_O&G.pdf', 'HPCL_AR_O&G.pdf', 'HPCL_ECT_O&G.pdf', 'HPCL_IP_O&G.pdf', 'HPCL_SR_O&G.pdf', 'IOC_AR_O&G.pdf', 'IOC_ECT_O&G.pdf', 'IOC_IP_O&G.pdf', 'IOC_SR_O&G.pdf', 'MRE_AR_O&G.pdf', 'MRE_ECT_O&G.pdf', 'MRE_IP_O&G.pdf', 'RIL_AR_O&G.pdf', 'RIL_ECT_O&G.pdf', 'RIL_IP_O&G.pdf', 'RIL_SR_O&G.pdf']


In [9]:
# Match S3 PDFs with metadata
matched_pdfs = [pdf for pdf in pdf_files if pdf in metadata_dict]

print("Matched PDFs with metadata:", matched_pdfs)


Matched PDFs with metadata: ['ABA_AR_O&G.pdf', 'ADA_AR_O&G.pdf', 'ADA_ECT_O&G.pdf', 'ADA_IP_O&G.pdf', 'ADA_SR_O&G.pdf', 'BPCL_AR_O&G.pdf', 'BPCL_ECT_O&G.pdf', 'BPCL_IP_O&G.pdf', 'BPCL_SR_O&G.pdf', 'HPCL_AR_O&G.pdf', 'HPCL_ECT_O&G.pdf', 'HPCL_IP_O&G.pdf', 'HPCL_SR_O&G.pdf', 'IOC_AR_O&G.pdf', 'IOC_ECT_O&G.pdf', 'IOC_IP_O&G.pdf', 'IOC_SR_O&G.pdf', 'MRE_AR_O&G.pdf', 'MRE_ECT_O&G.pdf', 'MRE_IP_O&G.pdf', 'RIL_AR_O&G.pdf', 'RIL_ECT_O&G.pdf', 'RIL_IP_O&G.pdf', 'RIL_SR_O&G.pdf']


In [9]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 113.5 MB/s eta 0:00:0000:01


In [10]:
import fitz  # PyMuPDF
import os

def extract_text_with_pymupdf(pdf_s3_bucket, pdf_file_key):
    """
    Downloads a PDF from S3 and extracts text using PyMuPDF (MuPDF).
    """
    temp_pdf_path = "/tmp/temp_document.pdf"  # Temp file path

    # ✅ Step 1: Download PDF from S3
    try:
        s3_client.download_file(pdf_s3_bucket, pdf_file_key, temp_pdf_path)
    except Exception as e:
        print(f"❌ Failed to download {pdf_file_key} from S3: {str(e)}")
        return ""

    # ✅ Step 2: Open PDF and Extract Text
    try:
        doc = fitz.open(temp_pdf_path)
        extracted_text = ""

        for page in doc:
            extracted_text += page.get_text("text") + "\n"

        doc.close()
        return extracted_text.strip()

    except Exception as e:
        print(f"🚨 PyMuPDF failed to extract text from {pdf_file_key}: {str(e)}")
        return ""

    finally:
        # ✅ Step 3: Delete Temporary File
        if os.path.exists(temp_pdf_path):
            os.remove(temp_pdf_path)


In [11]:
import time

# Initialize Textract client
textract_client = boto3.client('textract')

def extract_graphs_from_pdf(pdf_s3_bucket, pdf_file_key):
    """
    Extracts tables and forms from PDFs where `doc_id_type = SR` using Textract.
    """
    print(f"📊 Extracting graphs and tables for {pdf_file_key} (SR Document)...")

    try:
        response = textract_client.start_document_analysis(
            DocumentLocation={'S3Object': {'Bucket': pdf_s3_bucket, 'Name': pdf_file_key}},
            FeatureTypes=["TABLES", "FORMS"]
        )
        job_id = response['JobId']

        # Wait for Textract to complete processing
        while True:
            status = textract_client.get_document_analysis(JobId=job_id)
            if status["JobStatus"] in ["SUCCEEDED", "FAILED"]:
                break
            print("⏳ Waiting for Textract to process document...")
            time.sleep(5)

        if status["JobStatus"] == "FAILED":
            print(f"❌ Textract failed for {pdf_file_key}")
            return ""

        # ✅ Extract tables and forms
        extracted_text = []
        for block in status['Blocks']:
            if block['BlockType'] in ['TABLE', 'CELL']:
                extracted_text.append(block.get('Text', ''))

        return "\n".join(extracted_text)

    except Exception as e:
        print(f"⚠️ Failed to extract graphs/tables from {pdf_file_key}: {str(e)}")
        return ""


In [25]:
print(extracted_text_data.keys())
    

dict_keys(['ABA_AR_O&G.pdf', 'ADA_AR_O&G.pdf', 'ADA_ECT_O&G.pdf', 'ADA_IP_O&G.pdf', 'ADA_SR_O&G.pdf', 'BPCL_AR_O&G.pdf', 'BPCL_ECT_O&G.pdf', 'BPCL_IP_O&G.pdf', 'BPCL_SR_O&G.pdf', 'HPCL_AR_O&G.pdf', 'HPCL_ECT_O&G.pdf', 'HPCL_IP_O&G.pdf', 'HPCL_SR_O&G.pdf', 'IOC_AR_O&G.pdf', 'IOC_ECT_O&G.pdf', 'IOC_IP_O&G.pdf', 'IOC_SR_O&G.pdf', 'MRE_AR_O&G.pdf', 'MRE_ECT_O&G.pdf', 'MRE_IP_O&G.pdf', 'RIL_AR_O&G.pdf', 'RIL_ECT_O&G.pdf', 'RIL_IP_O&G.pdf', 'RIL_SR_O&G.pdf'])


In [144]:
metadata_dict['ABA_AR_O&G.pdf']

{'Company name': 'Aban Offshore',
 'Document Name': 'ABA_AR_O&G',
 'Company ID (Ticker)': 'INE053A01029',
 'Document ID Type': 'AR',
 'Document Category ID': 'FIN'}

In [ ]:
query_kendra("what are trends in relaince")

In [163]:
import boto3
import json

# Initialize Kendra Client
kendra_client = boto3.client("kendra")

# Your Kendra Index ID
kendra_index_id = "0bee42e9-9d01-44f9-8e56-32bbfa32e928"

def list_kendra_documents():
    """
    Queries Amazon Kendra and retrieves indexed document IDs.
    """
    # Perform a broad search to retrieve all documents
    response = kendra_client.query(
        IndexId=kendra_index_id,
        QueryText="*",  # Wildcard search to list all documents
        PageSize=50  # Fetch up to 50 documents at a time
    )

    print("\n📌 **Indexed Document IDs in Kendra:**\n")
    
    if "ResultItems" in response:
        for item in response["ResultItems"]:
            print(f"- {item.get('DocumentId', 'Unknown')}")
    else:
        print("❌ No documents found in Kendra.")

# Run the function
list_kendra_documents()



📌 **Indexed Document IDs in Kendra:**



In [166]:
import boto3
import json

# Initialize Kendra Client
kendra_client = boto3.client("kendra")

# Your Kendra Index ID
kendra_index_id = "0bee42e9-9d01-44f9-8e56-32bbfa32e928"

def check_kendra_document_count():
    """
    Retrieves document count from Kendra Index using describe_index().
    """
    response = kendra_client.describe_index(
        Id=kendra_index_id
    )

    print("\n📌 **Kendra Index Details:**")
    print(f"  ✅ Status: {response.get('Status', 'Unknown')}")
    print(f"  📂 Indexed Documents: {response.get('DocumentCount', 'Unknown')}")
    print(f"  🔄 Last Updated: {response.get('LastUpdatedAt', 'Unknown')}")

# Run the function
check_kendra_document_count()




📌 **Kendra Index Details:**
  ✅ Status: ACTIVE
  📂 Indexed Documents: Unknown
  🔄 Last Updated: Unknown
